In [ ]:
import pandas as pd
import numpy as np
import random


In [ ]:
# Fungsi untuk membaca data dari file CSV dan memuatnya sebagai array
def load_data(file_path):
    data = pd.read_csv(file_path)
    data = data.loc[0:59, 'Kasus harian']
    data = data.astype(float)
    data = np.array(data)
    return data

In [ ]:
# Fungsi untuk membuat populasi awal dengan gen-gen acak antara 0.1 dan 0.9
def create_initial_population(pop_size, gene_per_pop):
    population = np.random.uniform(low=0.1, high=0.9, size=(pop_size, gene_per_pop))
    return population

In [ ]:
# Fungsi untuk menghitung prediksi berdasarkan gen individu
def predict_cases(population, data):
    predictions = []
    for individual in population:
        pred = individual[0] * data[-1] + individual[1] * data[-2] + individual[2]
        predictions.append(pred)
    return predictions

In [ ]:
# Fungsi untuk menghitung fitness berdasarkan error prediksi terhadap data aktual
def calculate_fitness(predictions, actual_data):
    fitness = []
    for pred in predictions:
        error = abs((pred - actual_data[-1]) / pred) if pred > 0 else abs(0)  # Menggunakan MAPE
        fitness.append(1 / (error + 0.01))
    return fitness

In [ ]:
# Fungsi untuk seleksi menggunakan Roulette Wheel Selection
def roulette_wheel_selection(population, fitness):
    total_fitness = sum(fitness)
    probabilities = [fit / total_fitness for fit in fitness]
    selected_indices = np.random.choice(len(population), size=len(population), p=probabilities)
    selected_individuals = [population[i] for i in selected_indices]
    return np.array(selected_individuals)


In [ ]:
# Fungsi crossover dengan single-point crossover
def crossover(parents, pc):
    children = []
    for i in range(0, len(parents), 2):
        if random.random() < pc:
            crossover_point = random.randint(1, len(parents[i]) - 1)
            child1 = np.concatenate((parents[i][:crossover_point], parents[i + 1][crossover_point:]))
            child2 = np.concatenate((parents[i + 1][:crossover_point], parents[i][crossover_point:]))
            children.extend([child1, child2])
        else:
            children.extend([parents[i], parents[i + 1]])
    return np.array(children)

In [ ]:
# Fungsi untuk mutasi dengan nilai gen acak antara 0.1 dan 0.9
def mutation(children, mutation_rate):
    for child in children:
        if random.random() < mutation_rate:
            mutation_point = random.randint(0, len(child) - 1)
            child[mutation_point] = random.uniform(0.1, 0.9)
    return children

In [ ]:
# Main program
file_path = 'dataset/dataCovidIndonesia.csv'
data = load_data(file_path)

pop_size = 50
gene_per_pop = 3
pc = 0.7
mutation_rate = 0.1
max_generations = 10  # Ubah ke 10 untuk jumlah maksimum iterasi

population = create_initial_population(pop_size, gene_per_pop)

for generation in range(max_generations):  # Jumlah iterasi terbatas
    predictions = predict_cases(population, data)
    fitness = calculate_fitness(predictions, data)

    selected_population = roulette_wheel_selection(population, fitness)
    offspring = crossover(selected_population, pc)
    offspring_mutated = mutation(offspring, mutation_rate)
    print(fitness)

    population = np.concatenate((population, offspring_mutated))

    # Evaluasi prediksi hari ke-60 dan ke-61 menggunakan individu terbaik
    best_individual = population[np.argmax(fitness)]
    hari60 = round((best_individual[0] * data[-1]) + (best_individual[1] * data[-2]) + best_individual[2])
    hari61 = round((best_individual[0] * hari60) + (best_individual[1] * data[-1]) + best_individual[2])

    print(f"Iterasi {generation + 1}: Prediksi jumlah kasus pada hari ke-60:", hari60)
    print(f"Iterasi {generation + 1}: Prediksi jumlah kasus pada hari ke-61:", hari61)


[3.989499654290338, 8.283930234893399, 1.2294627950786643, 4.284039368107669, 5.186003620995973, 9.33553373711666, 34.012858956448405, 6.889543272258092, 0.9678428233330216, 2.9461811767190857, 1.050491704349628, 5.126489416935522, 4.1975635939804645, 1.69464158869474, 76.21192807002507, 6.453813325265636, 3.8846252167359854, 2.705319963934501, 4.150103024057106, 3.2965351592841956, 6.990752034216213, 3.7399768337339276, 5.476161356561572, 24.095786290421334, 25.52519314875115, 6.144773497684934, 3.8120989650751405, 2.3464357638947675, 7.747213383575135, 1.3925819961332864, 1.4130504937625024, 6.503610358931749, 2.9743822138343154, 1.7430312713583687, 0.2905353764866181, 1.2646125270188648, 1.018523697852467, 2.958396411592039, 16.60951603241984, 2.5761983283650265, 5.865799729498747, 7.158332137086415, 5.2018735657255535, 13.982051045325347, 1.037849870579738, 2.659273171730347, 4.749951197283933, 5.953888734231535, 3.582413681761262, 7.4556184598042705]
Iterasi 1: Prediksi jumlah kas

In [ ]:
# Menghitung prediksi untuk hari ke-60 dan ke-61 menggunakan individu terbaik
best_individual = population[np.argmax(fitness)]
hari60 = round((best_individual[0] * data[-1]) + (best_individual[1] * data[-2]) + best_individual[2])
hari61 = round((best_individual[0] * hari60) + (best_individual[1] * data[-1]) + best_individual[2])

print("Prediksi jumlah kasus pada hari ke-60:", hari60)
print("Prediksi jumlah kasus pada hari ke-61:", hari61)

Prediksi jumlah kasus pada hari ke-60: 433
Prediksi jumlah kasus pada hari ke-61: 456
